# Agent

# 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# Google検索Agent

In [2]:
from langchain.agents import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name = "GoogleSearch", # 名前は半角英数字のみ
        func=search.run,
        description="Search Google for recent results."
    )
]

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1.2)

In [4]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [5]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [6]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({"input": "岸田総理の日米首脳会談の結果について教えてください"})



> Entering new AgentExecutor chain...

Invoking: `GoogleSearch` with `岸田総理の日米首脳会談の結果`


Apr 10, 2024 ... 関連リンク · 日米共同記者会見（官邸HP） · 岸田総理大臣のホワイトハウスでの歓迎式典出席（外務省HP） 別ウィンドウで開く · 日米首脳会談（外務省 ... Nov 16, 2023 ... 現地時間11月16日午後5時15分（日本時間17日午前10時15分）から約15分間、APEC首脳会議に出席するため米国サンフランシスコを訪問中の岸田文雄内閣 ... Apr 12, 2024 ... 岸田総理が米国から国賓待遇で招待を受け2024年4月8日～14日の予定で訪米している。4月10日にバイデン大統領との公式な日米首脳会談が行われ、会談後 ... Nov 15, 2023 ... APEC（アジア太平洋経済協力、Asia Pacific Economic Cooperation） · 岸田総理大臣の米国訪問（令和5年11月15日～19日） · APEC首脳会議 · IPEF首脳会合. Nov 17, 2023 ... 【NHK】岸田総理大臣は、訪問先のアメリカでバイデン大統領との首脳会談に臨み、中国への対応について、16日の米中首脳会談の結果など… Feb 8, 2024 ... 岸田文雄首相は9日、政府与党連絡会議であいさつし、諸般の事情が許せば、国賓待遇で米国を公式に訪問し4月10日にワシントンⅮCで日米首脳会談を ... Apr 11, 2024 ... 【NHK】アメリカを訪れている岸田総理大臣はバイデン大統領と会談し、自衛隊とアメリカ軍の指揮・統制の向上など、防衛協力を深めるとと… Apr 11, 2024 ... ... 日米両国がグローバルなパートナーとして真価を発揮すべき時だ」と述べた。 岸田首相は首脳会談前日の4月9日に米国商工会議所と米日経済協議会（USJBC ... Apr 10, 2024 ... 【NHK】アメリカを訪問している岸田総理大臣は、日米首脳会談に先立ってホワイトハウスで行われた歓迎式典であいさつし、国際社会はこれ… 2024年4月10日（米国時間）、ワシントンDCを公式訪問中の岸田首相はバイデン大統領と日米首脳会談

{'input': '岸田総理の日米首脳会談の結果について教えてください',
 'output': '岸田総理の日米首脳会談の結果として、2024年4月10日に岸田総理がバイデン大統領と会談を行い、その後に共同声明およびファクトシートが発表されたことが報告されています。具体的な内容については、別途報道などをご確認いただければと思います。'}

# 対話型Google検索Agent

https://python.langchain.com/docs/modules/memory/agent_with_memory

In [8]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_openai import ChatOpenAI

In [9]:
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="現在の出来事に関する質問に答える必要がある場合に便利です",
    )
]

In [10]:
prefix = """あなたは日本語で回答します。あなたは人間と会話し、次の質問にできる限り答えてください。次のツールにアクセスできます:"""

suffix = """開始！"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [11]:
llm_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1.2), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

/Users/ohtakazuki/LLMProgramming/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


In [12]:
agent_chain.run(input="日本の人口は何人ですか？")

/Users/ohtakazuki/LLMProgramming/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I should use the search tool to find the most recent population data for Japan
Action: Search
Action Input: "日本の人口　2021"
Observation: Apr 15, 2022 ... 人口推計（2021年（令和3年）10月1日現在）結果の要約 · 総人口は1億2550万2千人で、前年に比べ64万4千人（‐0.51％）の減少となり、減少幅は比較可能な1950 ... Sep 16, 2022 ... 別表 日本における外国人の人口動態、外国における日本人の人口動態、前年以前発生分の人口動態 [41KB]; 人口 諸率の算出に用いた人口 [86KB]. PDF ... 表1 総人口及び日本人人口の推移(2015年～2021年). 注1) 2020年までの増減数には補間補正数（平成27年（2015年）国勢調査人口を基に算出した人口推計と、. 令和2年 ... 人口の多い国. 順位 ( じゅんい ), 国名, 推計 ( すいけい ) 人口 （1,000人）（2021年） ... 日本の国旗, 日本, 125,502. 注記 ( ちゅうき ). 出典 ... 人口の少ない国 · 世界 ... Apr 15, 2022 ... 2021年は64万4千人の減少となり、減少幅. は比較可能な1950年以降過去最大となった。 日本人人口は1億2278万人で、前年に比べ61万8千人（-0.50％）の ... 今回の推計では、2020年までの実績値をもとに、2020年10月1日現在の男女別年齢各歳別人口（総人口）を基準人口として、 2021年から2070年までの人口について推計しました ... The Statistics Bureau and the Director-General for Policy Planning of Japan play the central role in the official statistical system in producing and ... Aug 31, 2021 ... 「世界人口白書2021」によると、世界の総人口

'日本の人口は1億2550万2千人です（2021年時点）'

In [13]:
agent_chain.run(input="「その国の国歌を教えて")



> Entering new AgentExecutor chain...
Thought: I should use the Search tool to find the national anthem of Japan
Action: Search
Action Input: "日本の国歌"
Observation: 君が代 ... 『君が代』（きみがよ）は、日本の国歌である。10世紀初頭における最初の勅撰和歌集である『古今和歌集』の「読人知らず」の和歌を初出としている。世界の国歌の ... 日本の国歌はもちろん「君が代」ですね。 出典は、醍醐天皇の勅命により905年に奏上された「古今和歌集」の中の祝賀歌です。祝いの歌として1000年もの間、歌 ... この「君が代」は、海軍軍楽隊のお雇い教師、ドイツ人のF. エッケルト(Franz Eckert, 1852-1916)によって吹奏楽用に編曲され、明治21年に海軍省から各条約国に「大日本 ... 君が代(日本の国歌). M.s.. 1 SONG • 29 SECONDS • MAY 24 2018. Play. Purchase Options. 1. 君が代(日本の国歌). 00:29. ℗© 2018 A&S Ringtone. Top of page. Aug 30, 2012 ... 日本の国歌難しいよ…合唱団のうまい人とかに任せればいいのにと思ってしまう RT @dark_site_: 国歌斉唱して欲しいです. 君が代（きみがよ）は、日本の国歌。「天皇の治世」を奉祝する歌であり、「祝福を受ける人の寿命」を歌う和歌を元にしている。 歌詞は10世紀初めに編纂された『古今 ... Mar 31, 2021 ... ... 国歌が日本の競技場で流れるわけですから、今回はハイチ国歌について紹介したいと思います。 表のサイズを切り替える. ハイチ国旗. ハイチの国旗. ハイチ ... 君が代（きみがよ）は、日本の国歌。「天皇の治世」を奉祝する歌であり、「祝福を受ける人の寿命」を歌う和歌を元にしている。 歌詞は10世紀初めに編纂された『古今 ... Aug 24, 2021 ... そんな彼らが表彰台に上がってメダルを胸に飾る際、流れる音楽は何でしょうか？そう、その国のNational anthem（国歌）です

'「君が代」（きみがよ）'

# Index検索Agent

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# PDFの読込
loader = PyPDFLoader("./data2/001615363.pdf")
documents = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 検索（Retriever）の取得
retriever = db.as_retriever()

In [15]:
from langchain.tools.retriever import create_retriever_tool

tools = [
  create_retriever_tool(
      retriever,
      "ForeignTravelersQASystem",
      "訪日外国人の旅行や土産品、消費動向に関する質問に役立ちます。",
  )
]

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1.2)

In [17]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [18]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [19]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [20]:
agent_executor.invoke({"input": "お土産の購入状況を教えて"})



> Entering new AgentExecutor chain...

Invoking: `ForeignTravelersQASystem` with `{'query': 'お土産の購入状況'}`


訪日外国人消費動向調査  
 
20 ３．土産品の購入実態  
 
（１） 費目別購入率 
 費目別の 購入率（その費目を購入した人の
割合）は「菓子類」 （ 70.4％）、「その他食
料品・飲料・たばこ」 （ 43.4％）、 「衣類」
（33.8％）の順で高 い（図表3-1）。 
 国籍・地域別で 最も購入率が 高い費目は 、
韓国、台湾、香港 、中国で「菓子類」 （ それ
ぞれ82.9％、72.9％、63.3％、73.1％）、
米国で「その他食料品・飲料・たばこ」
（54.1％）となっている 。  
 
（２）費目別購入者単価  
 費目別の購入者単価 （その費目を購入し
た人における当該費目の 1人当たり平均
支出）は、「 宝石・貴金属 」が8.3万円
と最も高い（図表 3-1）。 
 国籍・地域別では 、台湾と香港の「宝
石・貴金属 」（それぞれ 19.9万円、
16.6万円）が他の国籍・地域に比べ高
い。
  図表3-1 買物代の費目別購入率および購入者単価  （主要国籍･地域別 、全目的）

訪日外国人消費動向調査 
 
38 
 （３）買物場所 
百貨店・デパート 
原則として百貨店協会加盟の店舗 
家電量販店 
PCやカメラ、電気製品を専門に販売する店舗 
ファッション専門店 
服・鞄・靴・アクセサリー・高級腕時計等
を専門に販売する店舗 
100 円ショップ 
店内の商品を原則として 1 点 100 日本円均
一で 販売する形態の小売店 
高速道路の SA・道の駅 
高速道路のサービスエリア（SA）やパーキ
ングエリア（PA）、国土交通省に登録され
ている「道の駅」 
観光地の土産店 
観光地内にある小売店 
宿泊施設 
旅館・ホテル等の館内に併設されている土産店 
スーパーマーケット 
高頻度に消費される食料品や日用品等をセル
フサービスで短時間に買えるようにした小売
店舗 
コンビニエンスストア 
年中無休で長時間の営業を行い、小規模な
店舗において主に食品、日用雑貨等多数の
品種を扱う形態の小売店 
ドラッグストア 
医

{'input': 'お土産の購入状況を教えて',
 'output': '訪日外国人のお土産の購入状況について以下の情報があります。\n \n20 ３．土産品の購入実態\n（１） 費目別購入率:\n- 費目別の購入率は「菓子類」が70.4％で最も高く、次いで「その他食料品・飲料・たばこ」（43.4％）、「衣類」（33.8％）となっています。\n\n（２） 費目別購入者単価:\n- 購入者単価が最も高いのは「宝石・貴金属」で8.3万円です。国籍・地域別では台湾と香港が他の地域に比べて購入者単価が高いことがわかります。\n\n（３） 買物場所:\n- 買物場所は百貨店・デパート、家電量販店、ファッション専門店、100 円ショップなどがあります。\n\nさらに、国籍・地域別の費目別購入率および購入者単価の情報が提供されています。\n\nこれらの情報を基に、特定のお土産の購入状況について詳細を知りたい場合はお知らせください。'}

# OpenAI Tool

In [21]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [22]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

In [23]:
import requests
from bs4 import BeautifulSoup
from langchain.chains import TransformChain

# ニュースを検索する関数
def get_news(inputs: dict) -> dict:
    keyword = inputs["keyword"]

    # WebサイトのURLを指定
    url = f'https://news.google.com/search?q={keyword}&hl=ja&gl=JP&ceid=JP:ja'

    # Requestsを利用してWebページを取得する
    response = requests.get(url)
    html = response.text

    # BeautifulSoupを利用してWebページを解析する
    soup = BeautifulSoup(html, 'html.parser')

    # soup.selectを利用して、ニュースのタイトルを取得する
    elements = soup.select('article div div div a')

    # ニュースのトップ3を配列に取得
    news = []
    for i, element in enumerate(elements):
        news.append({"title": element.getText()})
        if i>=2:
            break

    return {"result": news}

# 関数をChain化
news_chain = TransformChain(
    input_variables=["keyword"],
    output_variables=["result"],
    transform=get_news
)


In [24]:
# 数式の文字列をもとに計算を行う関数
def calc_func(inputs: dict) -> dict:
    formula = inputs["formula"]
    answer = eval(formula)
    return {"answer": f"{answer}"}

# 関数をChain化
calc_chain = TransformChain(
    input_variables=["formula"],
    output_variables=["answer"],
    transform=calc_func
)

In [25]:
from langchain.agents import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name = "news", # 半角英数字のみ
        func=news_chain.run,
        description="ニュースを探す際に役立ちます"
    ),
    Tool(
        name = "calculation", # 半角英数字のみ
        func=calc_chain.run,
        description="計算をする場合に役立ちます"
    ),
    Tool(
        name = "Search", # 半角英数字のみ
        func=search.run,
        description="現在の出来事や世界の現状に関する質問に答える必要がある場合に役立ちます"
    ),
]

In [26]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [27]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [28]:
agent_executor.invoke({"input": "LangChainとは何ですか?"})



> Entering new AgentExecutor chain...

Invoking: `Search` with `LangChain`


From startups to global enterprises, ambitious builders choose LangChain. Build. LangChain gives developers a framework to construct LLM‑powered apps easily. What is LangChain? · Open-source libraries: Build your applications using LangChain's modular building blocks and components. · Productionization: Inspect, ... LangChain is a framework for developing applications powered by large language models (LLMs). Introduction. LangChain is a framework for developing applications powered by language models. It enables applications that: ... This framework consists of ... Quickstart · Get setup with LangChain, LangSmith and LangServe · Use the most basic and common components of LangChain: prompt templates, models, and output ... langchain.chains ¶. Chains are easily reusable components linked together. Chains encode a sequence of calls to components like models, document retrievers, ... LangChain is a framework de

{'input': 'LangChainとは何ですか?',
 'output': 'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションの開発フレームワークです。LangChainは、オープンソースのライブラリを提供し、モジュラーな構築ブロックやコンポーネントを使用してアプリケーションを構築することができます。また、LangChainはPythonとJavaScriptで書かれており、チャットボットや仮想アシスタントなどのアプリケーションの開発を簡素化するために設計されています。'}

In [29]:
agent_executor.invoke({"input": "8掛ける12は？"})



> Entering new AgentExecutor chain...

Invoking: `calculation` with `8*12`


968 かける 12 は 96 です。

> Finished chain.


{'input': '8掛ける12は？', 'output': '8 かける 12 は 96 です。'}

In [30]:
agent_executor.invoke({"input": "夏祭りに関するニュースを教えてください"})



> Entering new AgentExecutor chain...

Invoking: `news` with `夏祭り`


[{'title': '【U-NEXT×THE ALFEE】今夏開催の2DAYSイベント「THE ALFEE 2024 Wind of Time ５０年目の夏祭り」にU-NEXT ...'}, {'title': 'THE ALFEE、デビュー50年目の“夏祭り”開催 8月に『夏イベ』Kアリーナ横浜2Days'}, {'title': '《ファイターズ 超夏祭り》高さ12mの巨大ウォータースライダー登場！おとなから子どもまで夏を「超」満喫しよう ...'}]夏祭りに関するニュースですね。以下は最新の夏祭りに関するニュースの一部です：
1. 【U-NEXT×THE ALFEE】今夏開催の2DAYSイベント「THE ALFEE 2024 Wind of Time ５０年目の夏祭り」にU-NEXT ...
2. THE ALFEE、デビュー50年目の“夏祭り”開催 8月に『夏イベ』Kアリーナ横浜2Days
3. 《ファイターズ 超夏祭り》高さ12mの巨大ウォータースライダー登場！おとなから子どもまで夏を「超」満喫しよう ...

どのニュースに興味がありますか？

> Finished chain.


{'input': '夏祭りに関するニュースを教えてください',
 'output': '夏祭りに関するニュースですね。以下は最新の夏祭りに関するニュースの一部です：\n1. 【U-NEXT×THE ALFEE】今夏開催の2DAYSイベント「THE ALFEE 2024 Wind of Time ５０年目の夏祭り」にU-NEXT ...\n2. THE ALFEE、デビュー50年目の“夏祭り”開催 8月に『夏イベ』Kアリーナ横浜2Days\n3. 《ファイターズ 超夏祭り》高さ12mの巨大ウォータースライダー登場！おとなから子どもまで夏を「超」満喫しよう ...\n\nどのニュースに興味がありますか？'}

# Toolkit:Pandas DataFrame Agent

In [31]:
import pandas as pd

df = pd.read_csv("./data2/iris.data", header=0)
df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [32]:
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [33]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [34]:
agent.run("何行ありますか？")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


150データフレームには150行あります。

> Finished chain.


'データフレームには150行あります。'

In [35]:
agent.run("Nameの基本統計量を出力してください")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Name'].describe()"}`


count             150
unique              3
top       Iris-setosa
freq               50
Name: Name, dtype: objectName列の基本統計量は以下の通りです:
- データ数: 150
- ユニークな値の数: 3
- 最頻値: Iris-setosa
- 最頻値の出現回数: 50

> Finished chain.


'Name列の基本統計量は以下の通りです:\n- データ数: 150\n- ユニークな値の数: 3\n- 最頻値: Iris-setosa\n- 最頻値の出現回数: 50'

In [36]:
agent.run("このデータについて説明してください")



> Entering new AgentExecutor chain...
このデータは、アヤメの花（Iris）のがく片（Sepal）と花弁（Petal）の長さと幅を記録したものです。各行は1つのアヤメの花を表し、それぞれの花についてがく片と花弁の長さと幅が記録されています。また、各花の品種（Name）も記録されています。データの先頭から5行を示しています。

> Finished chain.


'このデータは、アヤメの花（Iris）のがく片（Sepal）と花弁（Petal）の長さと幅を記録したものです。各行は1つのアヤメの花を表し、それぞれの花についてがく片と花弁の長さと幅が記録されています。また、各花の品種（Name）も記録されています。データの先頭から5行を示しています。'

# Toolkit:Python Agent

In [37]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

In [38]:
tools = [PythonREPLTool()]

In [39]:
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

In [40]:
instructions = """あなたは、質問に答えるために Python コードを作成して実行するように設計されたエージェントです。
Python コードを実行するために使用できる Python REPL にアクセスできます。
エラーが発生した場合は、コードをデバッグして再試行してください。
コードの出力は質問に答えるためにのみ使用してください。
コードを実行しなくても答えがわかる場合もありますが、答えを得るにはコードを実行する必要があります。
質問に答えるコードを書けそうにない場合は、「わかりません」と答えてください。
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [41]:
agent = create_openai_functions_agent(ChatOpenAI(temperature=0), tools, prompt)

In [42]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [43]:
agent_executor.invoke({"input": "100までの素数を求めよ"})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.



Invoking: `Python_REPL` with `def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = [num for num in range(2, 101) if is_prime(num)]
primes`


100までの素数は、\[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97\] です。

> Finished chain.


{'input': '100までの素数を求めよ',
 'output': '100までの素数は、\\[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97\\] です。'}